# Experiment

In [ ]:
# Script for generate  experiments
# Imports
from classes.CBN import CBN
import ray
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Ray Configurations
ray.init(num_cpus=12, log_to_driver=False)

# Experiment for RDD variation
n_experiments = 10
l_experiments = []
for cont_experiment in range(1,n_experiments+1):
    print("============================")
    print("Experiment:", cont_experiment)
    print("============================")

    # Variable Parameters
    n_rddas_min = 3
    n_rddas_max = 20

    # Fixed Parameters
    n_of_variables_rdda = 5
    n_of_signals_rdda = 2
    n_exit_variables = 2
    n_clauses_function = 2
    type_network = "ALEATORY"
    # this name has to be unique
    unique_path = "22_11_2023"

    # List of Result for the Experiments
    l_res_experiment = []

    v_n_network = 1
    for n_of_rdds in range(n_rddas_min, n_rddas_max + 1):
        print("Number of Network:", v_n_network)
        print("-------------------------------")

        path = "files/" + unique_path + "_" + str(n_of_rdds) + "_" + str(n_of_variables_rdda) + "_" + str(n_of_signals_rdda) \
               + "_" + str(n_exit_variables) + "_" + str(n_clauses_function)

        # generate the RDDAs of the Network of RDDAs
        print("generating the Network of RDDAs ...")
        o_cbn = CBN(n_of_rdds, n_of_variables_rdda, n_of_signals_rdda, n_exit_variables,
                    n_clauses_function)

        # Generate the RDDs
        print("generating the rdds ...")
        o_cbn.generate_rddas(type_network=type_network)

        # # Save the Network of RDDAs in a Pickle file
        # CBN.save_file_pickle(oCBN, path)
        # path += ".pickle"
        #
        # print("=======================================================")
        # print("The Network of RDDAs is saved in: ", path)

        # Show the Network of RDDAs
        # oCBN.show()

        # Calculate the Attractors by RDDA and by Signal
        v_begin_0 = time.time()
        result = CBN.find_attractors_rddas_ray.remote(o_cbn)
        o_cbn = ray.get(result)
        v_end_0 = time.time()
        v_time_0 = v_end_0 - v_begin_0

        # Calculate the Attractors by RDDA and by Signal
        v_begin_1 = time.time()
        result = CBN.calculation_compatible_pairs_ray.remote(o_cbn)
        o_cbn = ray.get(result)
        v_end_1 = time.time()
        v_time_1 = v_end_1 - v_begin_1

        # # Calculate the Attractors by RDDA and by Signal with iterative Method
        # v_begin_1 = time.time()
        # # result = CBN.assembly_attractor_fields_iterative.remote(oCBN)
        # # oCBN = ray.get(result)
        # v_end_1 = time.time()
        # v_time_1 = v_end_1 - v_begin_1

        # Calculate the Attractors by RDDA and by Signal with optimized Method
        v_begin_2 = time.time()
        result = CBN.assembly_attractor_fields_optimized.remote(o_cbn)
        o_cbn = ray.get(result)
        v_end_2 = time.time()
        v_time_2 = v_end_2 - v_begin_2

        # Save the results for the experiment , numeric and time indicators
        res_dict = {
                    "n_network": v_n_network,
                    "n_rdds": n_of_rdds,
                    "n_rdda_attractors": len(o_cbn.d_global_rdda_attractor.items()),
                    "t_find_attractors_method": v_time_0,
                    "n_pair_attractors": len(o_cbn.list_attractors_pairs),
                    "t_comp_paris_method": v_time_1,
                    "n_attractor_fields": len(o_cbn.attractor_fields),
                    "t_optimized_method": v_time_2
                    }
        l_res_experiment.append(res_dict)
        v_n_network = v_n_network + 1

    # Read  the Data
    df = pd.DataFrame.from_dict(l_res_experiment)
    l_experiments.append(df)
print("END EXPERIMENT")

# Analysing the result

### Time vs RDDAs

In [ ]:
# Show the graphs for every experiment
l_labels_y = ["Number of attractors", "Time of find Attractors", "Number of pairs attractors", "Time to compute the attractor pairs", "Number of attractor Fields", "Time of find fields attractors"]
n_experiment = 1
for df in l_experiments:
    fig_, axs = plt.subplots(1,6,figsize=(24,5))
    cont_label = 0
    for (ax, y) in zip(axs, np.asarray(df.keys())[2:]):
        ax.plot(df['n_rdds'], df[y])
        ax.set_xlabel('Number of RDDs by RDDA')
        ax.set_ylabel(l_labels_y[cont_label])
        ax.grid(True)
        cont_label += 1
    fig_.suptitle('Result of the experiment: ' + str(n_experiment), fontsize=16)
    fig_.tight_layout()
    n_experiment +=1

In [ ]:
# Boxplot analysis
l_labels_y = ["Number of attractors", "Time of find Attractors", "Number of pairs attractors", "Time to compute the attractor pairs", "Number of attractor Fields", "Time of find fields attractors"]
n_experiment = 1
for df in l_experiments:
    # Filter the data to show
    
    # Create the Boxplot
    fig1, ax1 = plt.subplots()
    ax1.set_title('Number of Attractors')
    ax1.boxplot(df['n_rdda_attractors'])




In [ ]:
# Analysing the data
for df in l_experiments:
    print(df)

In [ ]:
# Save the result of the experiments in one csv file

In [ ]:
# Save tthe result of the experiment in one pickled file

In [ ]:
# # Show the graphs for every experiment
# l_labels_y = ["Number of attractors", "Time of find Attractors", "Number of pairs attractors", "Time to compute the attractor pairs", "Number of attractor Fields", "Time of find fields attractors"]
# n_experiment = 1
# for df in l_experiments:
#     fig_, axs = plt.subplots(1,6,figsize=(24,5))
#     cont_label = 0
#     for (ax, y) in zip(axs, np.asarray(df.keys())[2:]):
#         ax.plot(df['n_rdds'], df[y])
#         ax.set_xlabel('Number of RDDs by RDDA')
#         ax.set_ylabel(l_labels_y[cont_label])
#         ax.grid(True)
#         cont_label += 1
#     fig_.suptitle('Result of the experiment: ' + str(n_experiment), fontsize=16)
#     fig_.tight_layout()
#     n_experiment +=1

# for v_reg_dict in l_res_experiment:
#     # Show the metrics
#     print("===========================================")
#     print("Result of Network:", v_reg_dict["n_network"])
#     print("===========================================")
#     print("Number of RDDs:", v_reg_dict["n_rdds"])
#     print("Number of Attractors :", v_reg_dict["n_rdda_attractors"])
#     print("Number of Attractor Fields:", v_reg_dict["n_attractor_fields"])
#     print("Duration Compatible Pairs Method :", v_reg_dict["t_comp_paris_method"])
#     # print("Duration Iterative Method :", v_reg_dict["t_iterative_method"])
#     print("Duration Optimized Method:", v_reg_dict["t_optimized_method"])
